In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier

# Чтение данных

In [2]:
train = pd.read_csv('train.csv').set_index("user_id")
train

,target,time
user_id,,
3,0,77
13,0,86
37,0,89
41,0,57
42,0,84
...,...,...
561824,0,91
562043,0,75
562312,0,91


In [3]:
report_dates = pd.read_csv('report_dates.csv', parse_dates=['report_dt']).set_index("report")
#report_dates["report_dt"] = report_dates["report_dt"].astype('datetime64[ns]')
#report_dates["report_dt"] = report_dates["report_dt"]-pd.Timedelta("100 days")
report_dates["report_dt"] = report_dates["report_dt"].dt.date.astype('datetime64[ns]')
report_dates

,report_dt
report,
1,2022-07-31
2,2022-08-31
3,2022-09-30
4,2022-10-31
5,2022-11-30
6,2022-12-31
7,2023-01-31
8,2023-02-28
9,2023-03-31


In [4]:
%%time

clients = pd.read_csv('clients.csv').set_index("user_id")
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':0,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)
clients = clients.join(report_dates, on="report", how="left")
clients

CPU times: user 82.4 ms, sys: 31.1 ms, total: 113 ms
Wall time: 121 ms


<timed exec>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


,report,employee_count_nm,bankemplstatus,customer_age,report_dt
user_id,,,,,
3,2,4.0,0,3,2022-08-31
9,1,6.0,0,3,2022-07-31
13,6,5.0,0,2,2022-12-31
37,5,6.0,0,2,2022-11-30
41,1,4.0,0,2,2022-07-31
...,...,...,...,...,...
562043,12,NaN,0,2,2023-06-30
562205,12,NaN,0,1,2023-06-30
562312,12,NaN,0,0,2023-06-30


In [5]:
%%time
transactions = pd.read_csv('transactions.csv.zip', 
                           parse_dates=['transaction_dttm'], 
                           low_memory=False, compression='zip')
transactions["transaction_dttm"] = transactions["transaction_dttm"].dt.date
transactions["transaction_amt"]=np.where(transactions["currency_rk"]>1,
                                         transactions["transaction_amt"]*70,
                                         transactions["transaction_amt"])
transactions

CPU times: user 24.1 s, sys: 1.85 s, total: 25.9 s
Wall time: 25.8 s


,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,3,3,1,-183.883957,2022-01-28
1,3,3,1,-3206.437012,2022-01-28
2,3,16,1,-153866.890625,2022-02-16
3,3,56,1,-15144.601562,2022-03-09
4,3,0,1,5297.908691,2022-03-12
...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20
13075019,562740,9,1,-187.658463,2023-03-20
13075020,562740,1,1,-891.933350,2023-03-20
13075021,562740,13,1,-464.467316,2023-03-20


# Расчет дополнительных свойств

In [6]:
def logsumabs(x):
    return (np.log(x.abs().sum()))
temp_tran = transactions.groupby(['user_id','currency_rk'])['transaction_amt'].agg([logsumabs])
temp_tran = temp_tran.unstack('currency_rk').fillna(0)
temp_tran.columns = ['logsumabs_0','logsumabs_1','logsumabs_2','logsumabs_3']
temp_tran

,logsumabs_0,logsumabs_1,logsumabs_2,logsumabs_3
user_id,,,,
3,0.00000,12.789712,0.0,0.0
9,0.00000,12.686752,0.0,0.0
13,9.28478,12.825245,0.0,0.0
37,0.00000,12.775178,0.0,0.0
41,0.00000,11.595303,0.0,0.0
...,...,...,...,...
562043,0.00000,10.294896,0.0,0.0
562205,0.00000,10.701664,0.0,0.0
562312,0.00000,9.827568,0.0,0.0


In [7]:
%%time
transactions['transaction_dttm'] = transactions['transaction_dttm'].astype('datetime64[ns]')
transactions['dweek'] = transactions['transaction_dttm'].dt.dayofweek# .day//8
transactions['date'] = transactions['transaction_dttm'].dt.date.astype('datetime64[ns]') # .day//8
transactions['sp'] = np.where( transactions['transaction_amt']>0,transactions['transaction_amt'],0)
transactions['sm'] = np.where( transactions['transaction_amt']<0,transactions['transaction_amt'],0)
transactions['ss'] = transactions['sp'] - transactions['sm'] 
transactions

CPU times: user 5.63 s, sys: 381 ms, total: 6.01 s
Wall time: 5.88 s


,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,dweek,date,sp,sm,ss
0,3,3,1,-183.883957,2022-01-28,4,2022-01-28,0.000000,-183.883957,183.883957
1,3,3,1,-3206.437012,2022-01-28,4,2022-01-28,0.000000,-3206.437012,3206.437012
2,3,16,1,-153866.890625,2022-02-16,2,2022-02-16,0.000000,-153866.890625,153866.890625
3,3,56,1,-15144.601562,2022-03-09,2,2022-03-09,0.000000,-15144.601562,15144.601562
4,3,0,1,5297.908691,2022-03-12,5,2022-03-12,5297.908691,0.000000,5297.908691
...,...,...,...,...,...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20,0,2023-03-20,0.000000,-2484.366211,2484.366211
13075019,562740,9,1,-187.658463,2023-03-20,0,2023-03-20,0.000000,-187.658463,187.658463
13075020,562740,1,1,-891.933350,2023-03-20,0,2023-03-20,0.000000,-891.933350,891.933350
13075021,562740,13,1,-464.467316,2023-03-20,0,2023-03-20,0.000000,-464.467316,464.467316


In [8]:
transactions = transactions.join(clients ,on="user_id", how="left")
#transactions['diff_days'] = (transactions['report_dt']-transactions['transaction_dttm']).dt.days
transactions['diff_days'] = transactions['report_dt']-transactions['transaction_dttm']
transactions['m'] = (transactions['report_dt'].dt.year-
                     transactions['transaction_dttm'].dt.year)*12+(transactions['report_dt'].dt.month-
                                                                   transactions['transaction_dttm'].dt.month)
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,dweek,date,sp,sm,ss,report,employee_count_nm,bankemplstatus,customer_age,report_dt,diff_days,m
0,3,3,1,-183.883957,2022-01-28,4,2022-01-28,0.000000,-183.883957,183.883957,2,4.0,0,3,2022-08-31,215 days,7
1,3,3,1,-3206.437012,2022-01-28,4,2022-01-28,0.000000,-3206.437012,3206.437012,2,4.0,0,3,2022-08-31,215 days,7
2,3,16,1,-153866.890625,2022-02-16,2,2022-02-16,0.000000,-153866.890625,153866.890625,2,4.0,0,3,2022-08-31,196 days,6
3,3,56,1,-15144.601562,2022-03-09,2,2022-03-09,0.000000,-15144.601562,15144.601562,2,4.0,0,3,2022-08-31,175 days,5
4,3,0,1,5297.908691,2022-03-12,5,2022-03-12,5297.908691,0.000000,5297.908691,2,4.0,0,3,2022-08-31,172 days,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20,0,2023-03-20,0.000000,-2484.366211,2484.366211,12,NaN,0,0,2023-06-30,102 days,3
13075019,562740,9,1,-187.658463,2023-03-20,0,2023-03-20,0.000000,-187.658463,187.658463,12,NaN,0,0,2023-06-30,102 days,3
13075020,562740,1,1,-891.933350,2023-03-20,0,2023-03-20,0.000000,-891.933350,891.933350,12,NaN,0,0,2023-06-30,102 days,3
13075021,562740,13,1,-464.467316,2023-03-20,0,2023-03-20,0.000000,-464.467316,464.467316,12,NaN,0,0,2023-06-30,102 days,3


In [9]:
count_trans = transactions.groupby(['user_id']
    ).agg({
        'transaction_amt':[
            "sum",
            "max",
            "min",
          #  "median",
            "count",
            #"last"
            ],
        "mcc_code":[
            #"last",
            "nunique"
            ],
        "currency_rk":[
            "min",
            "max"
            ],
        'date':[
            "min",
            "max",
            "nunique",
            "count"
            ],
        "sp":"sum",
        "sm":"sum",
        "ss":"sum"
        })
count_trans.columns = count_trans.columns.map('_'.join).map(lambda x: "count_trans_"+str(x))
count_trans["count_trans_dates"]=(count_trans["count_trans_date_max"]-count_trans["count_trans_date_min"]).dt.days
count_trans

,count_trans_transaction_amt_sum,count_trans_transaction_amt_max,count_trans_transaction_amt_min,count_trans_transaction_amt_count,count_trans_mcc_code_nunique,count_trans_currency_rk_min,count_trans_currency_rk_max,count_trans_date_min,count_trans_date_max,count_trans_date_nunique,count_trans_date_count,count_trans_sp_sum,count_trans_sm_sum,count_trans_ss_sum,count_trans_dates
user_id,,,,,,,,,,,,,,,
3,13706.416641,104011.960938,-153866.890625,11,4,1,1,2022-01-28,2022-05-14,8,11,186108.229797,-172401.813156,358510.042953,106
9,-323434.666813,-45.579891,-90147.617188,90,22,1,1,2021-10-20,2022-04-19,54,90,0.000000,-323434.666813,323434.666813,181
13,-124717.379150,70322.828125,-58740.300781,22,4,0,1,2022-03-23,2022-09-07,18,22,128766.684326,-253484.063477,382250.747803,168
37,-331859.599463,5487.140625,-35782.984375,315,28,1,1,2022-02-19,2022-08-17,129,315,10738.788574,-342598.388037,353337.176611,179
41,-108586.614166,-290.766998,-16841.208984,16,5,1,1,2021-11-16,2022-04-18,12,16,0.000000,-108586.614166,108586.614166,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,-29581.256115,-40.769005,-13020.519531,37,12,1,1,2022-10-06,2023-02-07,23,37,0.000000,-29581.256115,29581.256115,124
562205,-40491.908630,1595.461060,-6220.171387,151,15,1,1,2022-09-22,2023-03-20,85,151,1968.906334,-42460.814964,44429.721298,179
562312,-18537.821270,-28.292030,-1372.377075,56,7,1,1,2022-09-22,2023-02-27,39,56,0.000000,-18537.821270,18537.821270,158


In [10]:
# Сумма положительный транзакций
sum_trans_p = transactions[transactions['transaction_amt']>0].groupby(['user_id']
    ).agg({'transaction_amt':["count","mean"]})#.rename(columns={"transaction_amt":"sum_trans_p"})
sum_trans_p.columns = sum_trans_p.columns.map('_'.join).map(lambda x: "sump_"+str(x))
sum_trans_p

,sump_transaction_amt_count,sump_transaction_amt_mean
user_id,,
3,7,26586.889971
13,8,16095.835541
37,2,5369.394287
42,11,6616.334467
46,6,901.388145
...,...,...
561824,4,484.960381
561908,10,93679.223340
562205,3,656.302111


In [11]:
# Сумма отрицательных транзакций
sum_trans_m = transactions[transactions['transaction_amt']<0].groupby(['user_id']
    ).agg({'transaction_amt':["count","mean"]})#.rename(columns={"transaction_amt":"sum_trans_p"})
sum_trans_m.columns = sum_trans_m.columns.map('_'.join).map(lambda x: "summ_"+str(x))
sum_trans_m

,summ_transaction_amt_count,summ_transaction_amt_mean
user_id,,
3,4,-43100.453289
9,90,-3593.718520
13,14,-18106.004534
37,313,-1094.563540
41,16,-6786.663385
...,...,...
562043,37,-799.493409
562205,148,-286.897398
562312,56,-331.032523


In [12]:
msump = transactions.pivot_table(values="sp",                              
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msump.columns = list( map(lambda x: "msump"+str(x), msump.columns ) )
msump

,msump3,msump4,msump5,msump6,msump7,msump8,msump9
user_id,,,,,,,
3,137734.573242,43075.747864,5297.908691,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13,10805.421875,10952.112305,0.000000,86198.161377,10038.188965,10772.799805,0.000000
37,0.000000,0.000000,5487.140625,5251.647949,0.000000,0.000000,0.000000
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
562205,319.916626,0.000000,0.000000,0.000000,0.000000,0.000000,1648.989708
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
msumm = transactions.pivot_table(values="sm",
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msumm.columns = list( map(lambda x: "msumm"+str(x), msumm.columns ) )
msumm

,msumm3,msumm4,msumm5,msumm6,msumm7,msumm8,msumm9
user_id,,,,,,,
3,0.000000,0.000000,-15144.601562,-153866.890625,-3390.320969,0.000000,0.000000
9,-13289.485764,-104223.187939,-109800.056656,-54146.384140,-18500.321304,-12387.577271,-11087.653740
13,-16394.193359,-27095.248047,-27650.769531,-76186.732422,-33908.388672,-53024.051758,-19224.679688
37,-32719.820328,-41364.700729,-67081.604048,-53322.076138,-69769.034760,-50056.823341,-28284.328693
41,-8045.445801,-17708.008820,-30354.633301,-17462.199585,-18175.117676,-16841.208984,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,-142.056610,0.000000,-2984.231926,-11080.026783,-15374.940796,0.000000
562205,-10452.779190,-7546.495855,-6601.707232,-4821.073084,-6082.155629,-5607.360139,-1349.243835
562312,0.000000,-2644.634823,-1362.274536,-3165.544842,-3938.301605,-4495.630554,-2931.434910


In [14]:
msum = transactions.pivot_table(values="transaction_amt",
    index="user_id",
    columns=["m","currency_rk"],
    #aggfunc='sum',
    aggfunc='count',
    fill_value=0,)
msum.columns = list( map(lambda x: "msum"+str(x), msum.columns ) )
msum

,"msum(3, 0)","msum(3, 1)","msum(3, 2)","msum(3, 3)","msum(4, 0)","msum(4, 1)","msum(4, 2)","msum(4, 3)","msum(5, 0)","msum(5, 1)",...,"msum(7, 2)","msum(7, 3)","msum(8, 0)","msum(8, 1)","msum(8, 2)","msum(8, 3)","msum(9, 0)","msum(9, 1)","msum(9, 2)","msum(9, 3)"
user_id,,,,,,,,,,,,,,,,,,,,,
3,0,3,0,0,0,3,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,0,8,0,0,0,15,0,0,0,9,...,0,0,0,9,0,0,0,14,0,0
13,0,2,0,0,0,3,0,0,0,2,...,0,0,1,3,0,0,0,1,0,0
37,0,15,0,0,0,77,0,0,0,67,...,0,0,0,37,0,0,0,10,0,0
41,0,2,0,0,0,4,0,0,0,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0,0,0,0,0,1,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
562205,0,12,0,0,0,29,0,0,0,33,...,0,0,0,23,0,0,0,8,0,0
562312,0,0,0,0,0,10,0,0,0,5,...,0,0,0,14,0,0,0,7,0,0


In [15]:
smc = transactions.pivot_table(values="sm",
    index="user_id",
    columns=["m","currency_rk"],
    aggfunc='count',
    fill_value=0,)
smc.columns = list( map(lambda x: "smc"+str(x), smc.columns ) )
smc

,"smc(3, 0)","smc(3, 1)","smc(3, 2)","smc(3, 3)","smc(4, 0)","smc(4, 1)","smc(4, 2)","smc(4, 3)","smc(5, 0)","smc(5, 1)",...,"smc(7, 2)","smc(7, 3)","smc(8, 0)","smc(8, 1)","smc(8, 2)","smc(8, 3)","smc(9, 0)","smc(9, 1)","smc(9, 2)","smc(9, 3)"
user_id,,,,,,,,,,,,,,,,,,,,,
3,0,3,0,0,0,3,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,0,8,0,0,0,15,0,0,0,9,...,0,0,0,9,0,0,0,14,0,0
13,0,2,0,0,0,3,0,0,0,2,...,0,0,1,3,0,0,0,1,0,0
37,0,15,0,0,0,77,0,0,0,67,...,0,0,0,37,0,0,0,10,0,0
41,0,2,0,0,0,4,0,0,0,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0,0,0,0,0,1,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
562205,0,12,0,0,0,29,0,0,0,33,...,0,0,0,23,0,0,0,8,0,0
562312,0,0,0,0,0,10,0,0,0,5,...,0,0,0,14,0,0,0,7,0,0


In [16]:
spc = transactions.pivot_table(values="sp",
    index="user_id",
    columns="m",
    aggfunc='count',
    fill_value=0,)
spc.columns = list( map(lambda x: "spc"+str(x), spc.columns ) )
spc

,spc3,spc4,spc5,spc6,spc7,spc8,spc9
user_id,,,,,,,
3,3,3,2,1,2,0,0
9,8,15,9,23,12,9,14
13,2,3,2,6,4,4,1
37,15,77,67,54,55,37,10
41,2,4,3,4,2,1,0
...,...,...,...,...,...,...,...
562043,0,1,0,5,27,4,0
562205,12,29,33,20,26,23,8
562312,0,10,5,6,14,14,7


In [17]:
spdc = transactions.pivot_table(values="sp",
    index="user_id",
    columns="dweek",
    aggfunc='count',
    fill_value=0,)
spdc.columns = list( map(lambda x: "spdc"+str(x), spdc.columns ) )
spdc

spds = transactions.pivot_table(values="sp",
    index="user_id",
    columns="dweek",
    aggfunc='sum',
    fill_value=0,)
spds.columns = list( map(lambda x: "spds"+str(x), spds.columns ) )
spds

,spds0,spds1,spds2,spds3,spds4,spds5,spds6
user_id,,,,,,,
3,5083.818176,0.000000,0.000000,28335.613281,37991.929688,114696.868652,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13,20843.610840,10952.112305,0.000000,0.000000,96970.961182,0.000000,0.000000
37,0.000000,0.000000,0.000000,5487.140625,0.000000,5251.647949,0.000000
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
562205,0.000000,0.000000,0.000000,53.528648,1595.461060,319.916626,0.000000
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
smdc = transactions.pivot_table(values="sm",
    index="user_id",
    columns="dweek",
    aggfunc='count',
    fill_value=0,)
smdc.columns = list( map(lambda x: "smdc"+str(x), smdc.columns ) )
smdc

smds = transactions.pivot_table(values="sm",
    index="user_id",
    columns="dweek",
    aggfunc='sum',
    fill_value=0,)
smds.columns = list( map(lambda x: "smds"+str(x), smds.columns ) )
smds

,smds0,smds1,smds2,smds3,smds4,smds5,smds6
user_id,,,,,,,
3,0.000000,0.000000,-169011.492188,0.000000,-3390.320969,0.000000,0.000000
9,-73575.006897,-22025.790817,-12068.200882,-5015.756996,-24209.228561,-170599.176193,-15941.506466
13,-22091.132812,-24650.388672,-52853.843750,-5997.509766,-96384.850586,-17878.054688,-33628.283203
37,-42904.638344,-79845.543495,-47002.623840,-56788.511864,-28587.469574,-43567.516178,-43902.084742
41,-18852.887085,-23463.530762,-28168.941437,-11565.626953,-24716.487305,-1819.140625,0.000000
...,...,...,...,...,...,...,...
562043,-2115.068794,-1778.833473,-1404.176968,-4581.644516,-15148.450806,-769.264641,-3783.816917
562205,-1928.830556,-3430.759820,-3260.953653,-5024.823685,-6419.355495,-16460.974468,-5935.117287
562312,-2670.059166,-1222.038811,-2914.708450,-4018.634880,-2457.707897,-3825.585869,-1429.086197


In [19]:
# Максимальная дата до отчета, по клиентам
max_day_trans = transactions.groupby(['user_id'])[['diff_days']].max().rename(columns={"diff_days":"max_day_trans"})
max_day_trans

,max_day_trans
user_id,
3,215 days
9,284 days
13,283 days
37,284 days
41,257 days
...,...
562043,267 days
562205,281 days
562312,281 days


In [20]:
# Минимальная дата до отчета, по клиентам
min_day_trans = transactions.groupby(['user_id']
            )[['diff_days']].min().rename(columns={"diff_days":"min_day_trans"})
min_day_trans

,min_day_trans
user_id,
3,109 days
9,103 days
13,115 days
37,105 days
41,104 days
...,...
562043,143 days
562205,102 days
562312,123 days


In [21]:
mcc_trans = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns="mcc_code",
    aggfunc="sum",
    #aggfunc=["sum","median"],
    #aggfunc=[logsumabs],
    fill_value=0,)
mcc_trans.columns = list( map(lambda x: "mcc"+str(x), mcc_trans.columns ) )
mcc_trans

,mcc0,mcc1,mcc2,mcc3,mcc4,mcc6,mcc7,mcc8,mcc9,mcc10,...,mcc372,mcc382,mcc389,mcc392,mcc407,mcc412,mcc413,mcc424,mcc434,mcc449
user_id,,,,,,,,,,,,,,,,,,,,,
3,186108.229797,0.000000,0.000000,-3390.320969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,-20556.742374,-8324.033737,-192534.730225,-1664.872650,-2037.417374,0.000000,-5204.533508,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.000000,0.000000,0.000000,-160818.399902,0.000000,0.000000,-5997.509766,0.000000,10805.421875,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,10738.788574,-35943.518551,-26588.514793,-214788.286194,-126.869576,-653.359955,0.000000,-1031.058945,-26518.769531,-924.123138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.000000,-740.059052,0.000000,-70219.408691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0.000000,-7254.742481,0.000000,-13020.519531,0.000000,-1303.106201,-786.304443,-870.556900,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562205,-10868.047920,-12028.663424,-1157.922058,0.000000,-129.403091,-6800.897217,0.000000,-2645.133282,0.000000,-770.019485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562312,0.000000,-6974.722969,-352.799313,0.000000,-1372.377075,-2114.061485,0.000000,-988.508408,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
mcc_transc = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns="mcc_code",
    aggfunc='count',
    fill_value=0,)
mcc_transc.columns = list( map(lambda x: "mccc"+str(x), mcc_transc.columns ) )
mcc_transc

,mccc0,mccc1,mccc2,mccc3,mccc4,mccc6,mccc7,mccc8,mccc9,mccc10,...,mccc372,mccc382,mccc389,mccc392,mccc407,mccc412,mccc413,mccc424,mccc434,mccc449
user_id,,,,,,,,,,,,,,,,,,,,,
3,7,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,27,11,5,3,2,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,14,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
37,2,97,129,28,1,2,0,7,1,4,...,0,0,0,0,0,0,0,0,0,0
41,0,2,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0,19,0,1,0,1,3,4,0,0,...,0,0,0,0,0,0,0,0,0,0
562205,30,33,2,0,1,10,0,6,0,3,...,0,0,0,0,0,0,0,0,0,0
562312,0,24,3,0,1,6,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
mcc_transmc = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    #columns=["m","mcc_code"],
    columns=["mcc_code","m"],
    aggfunc=['sum','median'],
    #aggfunc='sum',
    fill_value=0,
)
mcc_transmc.columns = list( map(lambda x: "mccmc"+str(x), mcc_transmc.columns ) )
mcc_transmc

,"mccmc('sum', 0, 3)","mccmc('sum', 0, 4)","mccmc('sum', 0, 5)","mccmc('sum', 0, 6)","mccmc('sum', 0, 7)","mccmc('sum', 0, 8)","mccmc('sum', 0, 9)","mccmc('sum', 1, 3)","mccmc('sum', 1, 4)","mccmc('sum', 1, 5)",...,"mccmc('median', 412, 8)","mccmc('median', 413, 3)","mccmc('median', 413, 4)","mccmc('median', 413, 5)","mccmc('median', 413, 6)","mccmc('median', 413, 7)","mccmc('median', 413, 9)","mccmc('median', 424, 6)","mccmc('median', 434, 4)","mccmc('median', 449, 7)"
user_id,,,,,,,,,,,,,,,,,,,,,
3,137734.573242,43075.747864,5297.908691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5323.413376,-2728.293545,-172.254761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0.000000,0.000000,5487.140625,5251.647949,0.000000,0.000000,0.000000,-2098.237946,-7730.614010,-4955.007622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-290.766998,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562205,-7609.368988,-1207.410591,-1161.791405,-568.072906,-1511.607269,-458.786469,1648.989708,0.000000,-108.493008,-2985.051830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1993.233418,-353.968170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
%%time
date_trans = transactions.groupby("user_id").agg({"date":"unique"}).explode("date").reset_index().sort_values(["user_id","date"])
date_trans["interval"]=date_trans.groupby("user_id")["date"].diff()
date_trans = date_trans.dropna() 
date_trans["interval"]=date_trans["interval"].dt.days
date_trans = date_trans.groupby("user_id").agg({"interval":["last","max"]})
date_trans.columns = list( map(lambda x: "date_"+str(x), date_trans.columns ) )
date_trans

CPU times: user 31.9 s, sys: 632 ms, total: 32.5 s
Wall time: 32.5 s


,"date_('interval', 'last')","date_('interval', 'max')"
user_id,,
3,9,30
9,17,22
13,2,21
37,1,7
41,2,31
...,...,...
562043,54,54
562205,1,8
562312,2,37


# Итоговая сборка

In [25]:
df = clients.join(
                count_trans, on="user_id", how="left"
                #).join(sum_trans, on="user_id", how="left"
                #).join(sum_trans_p, on="user_id", how="left"
                #).join(sum_trans_m, on="user_id", how="left"
                ).join(temp_tran, on="user_id", how="left"
                #).join(max_day_trans, on="user_id", how="left"
                #).join(min_day_trans, on="user_id", how="left"
                ).join(msump, on="user_id", how="left"
                ).join(msumm, on="user_id", how="left"
                ).join(msum, on="user_id", how="left"
                ).join(spc, on="user_id", how="left"
                ).join(smc, on="user_id", how="left"
                #).join(spdc, on="user_id", how="left"
                #).join(smdc, on="user_id", how="left"
                #).join(spds, on="user_id", how="left"
                #).join(smds, on="user_id", how="left"
                ).join(mcc_trans, on="user_id", how="left"
                ).join(mcc_transc, on="user_id", how="left"
                #).join(mcc_transmc, on="user_id", how="left"
                ).join(date_trans, on="user_id", how="left"
                       
                ).join(train, on="user_id", how="left"
                )

df['time'] = df['time'].fillna(-1)
df['time'] = df['time'].astype(np.int32)
df['target'] = df['target'].fillna(-1)
df['target'] = df['target'].astype(np.int8)

df

,report,employee_count_nm,bankemplstatus,customer_age,report_dt,count_trans_transaction_amt_sum,count_trans_transaction_amt_max,count_trans_transaction_amt_min,count_trans_transaction_amt_count,count_trans_mcc_code_nunique,...,mccc407,mccc412,mccc413,mccc424,mccc434,mccc449,"date_('interval', 'last')","date_('interval', 'max')",target,time
user_id,,,,,,,,,,,,,,,,,,,,,
3,2,4.0,0,3,2022-08-31,13706.416641,104011.960938,-153866.890625,11,4,...,0,0,0,0,0,0,9.0,30.0,0,77
9,1,6.0,0,3,2022-07-31,-323434.666813,-45.579891,-90147.617188,90,22,...,0,0,0,0,0,0,17.0,22.0,-1,-1
13,6,5.0,0,2,2022-12-31,-124717.379150,70322.828125,-58740.300781,22,4,...,0,0,0,0,0,0,2.0,21.0,0,86
37,5,6.0,0,2,2022-11-30,-331859.599463,5487.140625,-35782.984375,315,28,...,0,0,0,0,0,0,1.0,7.0,0,89
41,1,4.0,0,2,2022-07-31,-108586.614166,-290.766998,-16841.208984,16,5,...,0,0,0,0,0,0,2.0,31.0,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,12,NaN,0,2,2023-06-30,-29581.256115,-40.769005,-13020.519531,37,12,...,0,0,0,0,0,0,54.0,54.0,0,75
562205,12,NaN,0,1,2023-06-30,-40491.908630,1595.461060,-6220.171387,151,15,...,0,0,0,0,0,0,1.0,8.0,-1,-1
562312,12,NaN,0,0,2023-06-30,-18537.821270,-28.292030,-1372.377075,56,7,...,0,0,0,0,0,0,2.0,37.0,0,91


# просмотр корреляции

In [26]:
%%time
#cor=df.corr().abs().unstack().sort_values(ascending=False).dropna().reset_index()
#cor[cor["level_0"]!=cor["level_1"]].head(60)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


# train_test_split

In [27]:
X = df[df['time']!=-1][df.columns[:-2]].copy()
y = df[df['time']!=-1][['target']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
X_train

,report,employee_count_nm,bankemplstatus,customer_age,report_dt,count_trans_transaction_amt_sum,count_trans_transaction_amt_max,count_trans_transaction_amt_min,count_trans_transaction_amt_count,count_trans_mcc_code_nunique,...,mccc389,mccc392,mccc407,mccc412,mccc413,mccc424,mccc434,mccc449,"date_('interval', 'last')","date_('interval', 'max')"
user_id,,,,,,,,,,,,,,,,,,,,,
380772,1,NaN,0,3,2022-07-31,-98045.028499,-9.110510,-6399.057129,163,11,...,0,0,0,0,0,0,0,0,1.0,13.0
248009,7,6.0,0,1,2023-01-31,265.832468,523.658447,-364.052734,15,3,...,0,0,0,0,0,0,0,0,3.0,31.0
11357,10,4.0,0,2,2023-04-30,-412629.283176,182346.890625,-153051.875000,479,36,...,0,0,0,0,0,0,0,0,3.0,11.0
87338,10,6.0,0,1,2023-04-30,-78570.531029,-4.502690,-4582.610352,101,17,...,0,0,0,0,0,0,0,0,1.0,24.0
460153,9,3.0,0,2,2023-03-31,-1341.754089,14587.627930,-14638.060547,15,2,...,0,0,0,0,0,0,0,0,4.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237286,6,6.0,0,1,2022-12-31,96337.004486,90323.328125,-109855.257812,188,33,...,0,0,0,0,0,0,0,0,1.0,5.0
4847,1,5.0,0,3,2022-07-31,-158876.900894,18805.158203,-26947.216797,170,27,...,0,0,0,0,0,0,0,0,3.0,15.0
334641,12,NaN,0,3,2023-06-30,-244767.816072,9369.488281,-22261.675781,110,11,...,0,0,0,0,0,0,0,0,2.0,19.0


# Подбор гиперпараметров catboost (iterations, learning_rate, depth, l2_leaf_reg) по графикам

In [28]:
%%time
cat = CatBoostClassifier(
    iterations=14000,    
    learning_rate= 0.00291, 
    depth=10,
    custom_metric=['AUC',"Accuracy","Precision","F1"], 
    eval_metric="AUC", 
    random_seed=63+9+4,#+4, 
    task_type="GPU", devices='0:1',  #закомментировать строку если отсутствует или настроен GPU 
)
#cat_features = ["bankemplstatus","customer_age", "employee_count_nm"]
cat_features = ["bankemplstatus","customer_age"]
cat.fit(X_train,y_train, eval_set=(X_test, y_test), cat_features=cat_features, verbose=False, plot=True)
cat.best_score_ #0.7766878604888916 -> 0,7768357456 на LB
#0.7781568765640259 +4
#0.7781765460968018 
#0.7780963182449341 -100
#0.7769784927368164 +4
#0.7768157124519348

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: /src/catboost/catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 804: forward compatibility was attempted on non supported HW

In [29]:
predict = cat.predict_proba(df[df['time']==-1][df.columns[:-2]])[:,1]
submit = df[df['time']==-1].reset_index()[['user_id']].copy()
submit['predict'] = predict
submit.to_csv(f'submission0010_{cat.best_score_["validation"]["AUC"]}.csv',index=False) 
submit

CatBoostError: There is no trained model to use predict_proba(). Use fit() to train model. Then use this method.

In [ ]:
#Значимость признаков
cat.get_feature_importance( prettified=True).head(60), cat.get_feature_importance( prettified=True).iloc[60:120], cat.get_feature_importance( prettified=True).iloc[-480:-420], cat.get_feature_importance( prettified=True).iloc[-420:-360], cat.get_feature_importance( prettified=True).iloc[-360:-300], cat.get_feature_importance( prettified=True).iloc[-300:-240], cat.get_feature_importance( prettified=True).iloc[-240:-180], cat.get_feature_importance( prettified=True).iloc[-180:-120], cat.get_feature_importance( prettified=True).iloc[-120:-60], cat.get_feature_importance( prettified=True).tail(60)

# Расчет по выбранным гиперпараметрам на полных данных

In [ ]:
X_pred = df[df['time']==-1][df.columns[:-2]].copy()
for i in range(20):
    cat = CatBoostClassifier(
        iterations=13000,    
        learning_rate= 0.00291, 
        depth=10,
        custom_metric=['AUC',"Accuracy","Precision","F1"], 
        eval_metric="AUC", 
        random_seed=63+9+i, 
        task_type="GPU", devices='0:1'  #закомментировать строку если отсутствует или настроен GPU 
    )
    cat_features = ["bankemplstatus","customer_age"]
    cat.fit(X,y, eval_set=(X_test, y_test), cat_features=cat_features, verbose=False, plot=True)
    print(i, cat.best_score_)
    
    predict = cat.predict_proba(X_pred)[:,1]
    submit = df[df['time']==-1].reset_index()[['user_id']].copy()
    submit['predict'] = predict
    submit.to_csv(f'submission0010_{i}.csv',index=False)    